# American Sign Language (ASL) Final Project W251 <br>Detectron2 Custom Dataset
___

Adapted from Roboflow Collab located [here](https://colab.research.google.com/drive/1-TNOcPm3Jr3fOJG8rnGT9gh60mHUsvaW#scrollTo=kc8MmgZugZWR).

Used on a `g4dn.2xlarge` AWS EC2 Instance with Nvidia Deep Learning AMI installed and within a Nvidia docker container `nvcr.io/nvidia/pytorch:21.09-py3`

Command Line Scripts Executed:

```shell
docker run --net=host --gpus=all --ipc=host -v ~/data:/data -p 8882:8888 -ti nvcr.io/nvidia/pytorch:21.09-py3 bash
jupyter lab --no-browser --ip=0.0.0.0 --allow-root
 ```


#### 1. Download Detectron Repository and any dependencies

In [ ]:
### install dependencies
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

!apt-get update # make sure system is updated and install python3-opencv
!DEBIAN_FRONTEND=noninteractive apt-get install -y python3-opencv

#### install model
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

In [ ]:
# import libraries
import torch, torchvision
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# standard libraries
import numpy as np
import cv2
import random
import os

#from google.colab.patches import cv2_imshow

# common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from detectron2.data.datasets import register_coco_instances
# Training 
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.config import get_cfg

print(torch.__version__, torch.cuda.is_available())

#### 2. Download Images in COCO format from Roboflow

In [ ]:
!curl -L "https://app.roboflow.com/ds/QLy1EzrO08?key=OSNpEWnqN6" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
register_coco_instances("my_dataset_train", {}, "train/_annotations.coco.json", "train")
register_coco_instances("my_dataset_val", {}, "valid/_annotations.coco.json", "valid")
register_coco_instances("my_dataset_test", {}, "test/_annotations.coco.json", "test")

#### 3. Train model using detectron packages

In [ ]:
# training module to use coco validation during training
class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)

cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001


cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 1500 #adjust up if val mAP is still rising, adjust down if overfit
cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.GAMMA = 0.05


cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 36 #your number of classes + 1

cfg.TEST.EVAL_PERIOD = 500


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Use tensorboard
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_test")
inference_on_dataset(trainer.model, val_loader, evaluator)

#### 4. Save a Pytorch model by saving model weights

In [ ]:
%ls ./output/

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("my_dataset_test", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("my_dataset_test")

In [ ]:
# !git clone https://github.com/NVIDIA-AI-IOT/torch2trt
# %cd torch2trt
# !python setup.py install
%!pip install timm

#### 5. Convert Model to Tensorrt
(Currently not possible onnx.optimizer module used to convert to ONNX is not available and torch2trt does not work with the input format for this model)

In [1]:
# import detectron2
# import onnx
# import cv2

# # import some common detectron2 utilities
# from detectron2 import model_zoo
# from detectron2.config import get_cfg
# #from detectron2.export import export_onnx_model, export_caffe2_model
# from detectron2.modeling import build_model
# from detectron2.checkpoint import DetectionCheckpointer
# import detectron2.data.transforms as T
# import torch
# from torch2trt import torch2trt
# import timm
# import time
# import os
# import numpy as np
# import torch.backends.cudnn as cudnn

In [2]:
# im = cv2.imread('train/0-0_jpg.rf.6267a69264abb1ca2948505799de1968.jpg')

In [3]:
# im.shape


In [4]:
# os.getcwd()

In [6]:
# cfg2 = get_cfg()
# # add project-specific config 
# cfg2.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
# # Find a model from detectron2's model zoo.
# cfg2.MODEL.WEIGHTS = "output/model_final.pth"
# # cfg.MODEL.DEVICE='cpu'

# # Build model and prepare input
# model = build_model(cfg2)
# model.eval()
# checkpointer = DetectionCheckpointer(model)
# checkpointer.load(cfg2.MODEL.WEIGHTS)
# aug = T.ResizeShortestEdge([cfg2.INPUT.MIN_SIZE_TEST, cfg2.INPUT.MIN_SIZE_TEST],
#                            cfg2.INPUT.MAX_SIZE_TEST)

# height, width = im.shape[:2]
# image = aug.get_transform(im).apply_image(im)
# image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
# inputs = {"image": image, "height": height, "width": width}
# image.size()



# inputs = [{"image": image}]

In [7]:
# !pip uninstall onnx
# !pip install git+git://github.com/onnx/onnx.git@master

In [8]:
# import torch
# import torchvision

# dummy_input = torch.randn(800, 800, device="cuda")

In [9]:

# torch_out = model(inputs)

In [10]:
# !pip install onnxruntime

In [11]:
# import detectron2.export

In [12]:
# ! pip install  onnxoptimizer

In [13]:
# exported_model = export_caffe2_tracing(cfg2, model, [inputs], output_folder = "output")

In [14]:
# Export to Onnx model export_onnx_model
# onnxModel = export_onnx_model(cfg2, model, [inputs])
# onnx.save(onnxModel, "/data/detectron2_final.onnx")